In [1]:
# from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
from transliterate import translit

In [4]:
from time import sleep

In [104]:
import re

### Avito

In [5]:
# Что первично можно выцепить на сайте:
# Город, тип дома, кол-во комнат

In [6]:
base_url = 'https://www.avito.ru/'

In [135]:
def get_similar_flats(city, street, n_rooms, house_type):
    base_url = 'https://www.avito.ru/'
    city =  re.findall(r'[a-z- ]+', translit(city.lower(), reversed=True))[0]
    street = street.lower()
    rooms = {
        1: '1-komnatnye',
        2: '2-komnatnye',
        3: '3-komnatnye',
        4: '4-komnatnye'
    }   
    house_types = {
        'Кирпичный': 'kirpichnyy_dom',
        'Панельный': 'panelnyy_dom',
        'Блочный': 'blochnyy_dom',
        'Монолитный': 'monolitnyy_dom',
        'Деревянный': 'derevyannyy_dom'
    }   
    search = base_url + city + '/kvartiry/prodam/' + rooms[n_rooms] + '/' + house_types[house_type] + '?q=' + '+'.join(street.split()) + '&s=104'
    print(search)
    html_doc =  requests.get(search)
    # Проверка на корректность запроса
    if html_doc.status_code == 429:
        print(html_doc)
    else:
        print(html_doc)
    # Первая страница поиска Авито
    soup = BeautifulSoup(html_doc.text, "html.parser")
    # Нахожу все подходящие (не рекомендации) квартиры на этой странице
    flats = soup.find('div', class_ = 'items-items-kAJAg')  # получаю список квартир вместе с рекламой и рекомендациями
    flats_act = flats.find_all(attrs={"data-marker": "item"})  # баблы страничек как массив

    # Получаю сайт с каждой квартирой отдельно
    flats_sites = []
    for flat in flats_act:
        flats_sites.append('https://www.avito.ru' + flat.find(attrs={"class":"iva-item-sliderLink-bJ9Pv"})['href'])
    # Пустой датафрейм, который буду возвращать
    df = pd.DataFrame({
                        'floor':None,
                        'all_floors':None,
                        'rooms':None,
                        'all_square':None,
                        'live_square':None,
                        'kitch_square':None,
                        'tall':None,
                        'repair':None,
                        'bathroom':None,
                        'balcony':None,
                        'window':None,
                        'dop_info':None,
                        'house_type':None,
                        'year_built':None,
                        'how_pas_lift':None,
                        'how_grz_lift':None,
                        'in_house':None,
                        'backyard':None,
                        'park':None,
                        'flat_lat':None,
                        'flat_lon':None,
                        'price':None
                        }, index = [0])

    for flat in flats_sites[:5]:
        flat_i = BeautifulSoup(requests.get(flat).text, "html.parser")
        sleep(5.5)
#        print(flat_i)
        flat_one_info = {
                        'floor':None,
                        'all_floors':None,
                        'rooms':None,
                        'all_square':None,
                        'live_square':None,
                        'kitch_square':None,
                        'tall':None,
                        'repair':None,
                        'bathroom':None,
                        'balcony':None,
                        'window':None,
                        'dop_info':None,
                        'house_type':None,
                        'year_built':None,
                        'how_pas_lift':None,
                        'how_grz_lift':None,
                        'in_house':None,
                        'backyard':None,
                        'park':None,
                        'flat_lat':None,
                        'flat_lon':None,
                        'price':None
                        }

        for text in flat_i.find_all('li', class_ = 'item-params-list-item'):
            label = text.text[:text.text.find(':')].split()[0]
            value = text.text[text.text.find(':') + 1:].split()
            if label == 'Этаж':
                flat_one_info['floor'] = value[0]
                flat_one_info['all_floors'] = value[2]
            elif label == 'Количество':
                flat_one_info['rooms'] = value[0]
            elif label == 'Общая':
                flat_one_info['all_square'] = value[0]
            elif label == 'Площадь':
                flat_one_info['live_square'] = value[0]
            elif label == 'Жилая':
                flat_one_info['kitchen_square'] = value[0]
            elif label == 'Высота':
                flat_one_info['tall'] = value[0]   
            elif label == 'Ремонт':
                flat_one_info['repair'] = value[0]
            elif label == 'Санузел':
                flat_one_info['bathroom'] = value[0]
            elif label == 'Балкон':
                flat_one_info['balcony'] = value[0]
            elif label == 'Вид':
                flat_one_info['window'] = ' '.join(value)
            elif label == 'Дополнительно':
                flat_one_info['dop_info'] = ' '.join(value)
            elif label == 'Тип':
                flat_one_info['house_type'] = value[0]
            elif label == 'Год':
                flat_one_info['year_built'] = value[0]
            elif label == 'Пассажирский':
                flat_one_info['how_pas_lift'] = value[0]
            elif label == 'Грузовой':
                flat_one_info['how_grz_lift'] = value[0]
            elif label == 'В':
                flat_one_info['in_house'] = ' '.join(value)
            elif label == 'Двор':
                flat_one_info['backyard'] = ' '.join(value)
            elif label == 'Парковка':
                flat_one_info['park'] = ' '.join(value) 
            flat_one_info['price'] = int(flat_i.find('span', class_ = 'js-item-price')['content'])
            
        flat_one_info['flat_lat'] = flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lat']
        flat_one_info['flat_lon'] = flat_i.find('div', class_ = 'b-search-map expanded item-map-wrapper js-item-map-wrapper')['data-map-lon']
        df = df.append(flat_one_info, ignore_index=True)
        print(' ---------- done! ----------')
    df.drop(index = 0,inplace=True)
    sleep(5.5)
    return df
# else:
#     print('Ошибка при поключении к сайту!')
#     print(html_doc)
#    return 0

In [137]:
res = get_similar_flats('Хабаровск', 'Гамарника', 1, 'Кирпичный')

https://www.avito.ru/habarovsk/kvartiry/prodam/1-komnatnye/kirpichnyy_dom?q=гамарника&s=104
<Response [429]>


AttributeError: 'NoneType' object has no attribute 'find_all'

In [123]:
res.to_excel('')

,floor,all_floors,rooms,all_square,live_square,kitch_square,tall,repair,bathroom,balcony,...,house_type,year_built,how_pas_lift,how_grz_lift,in_house,backyard,park,flat_lat,flat_lon,price
1,5,10,3,96.6,9,None,None,None,None,балкон,...,кирпичный,None,None,None,None,None,None,54.97715,60.353806,5500000


#### Получение координат из адреса по Гугл-картам

In [44]:
url = 'https://www.google.com/maps/search/' + 'Пензенская обл, г Кузнецк, ул 60-летия ВЛКСМ, д 8'

In [47]:
html_doc =  requests.get(url)

# Первая страница поиска Авито
soup = BeautifulSoup(html_doc.text, "html.parser")

In [88]:
source_lat = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[0])
source_lon = float(str(soup).split('?center=')[1].split('&amp;')[0].split('%2C')[1])